CLI command:

smartctl -A -i /dev/sda -j > disk_data.json

In [2]:
import json
import pandas as pd

def pripravi_podatke_za_model(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    nujni_stolpci = [
        'smart_1_raw', 'smart_3_raw', 'smart_4_raw', 'smart_5_raw',
        'smart_7_raw', 'smart_9_raw', 'smart_10_raw', 'smart_12_raw',
        'smart_187_raw', 'smart_188_raw', 'smart_190_raw', 'smart_191_raw',
        'smart_192_raw', 'smart_193_raw', 'smart_194_raw', 'smart_197_raw',
        'smart_198_raw', 'smart_199_raw'
    ]

    vsebina = {
        'model': data.get('model_name', 'Unknown'),
        'capacity_bytes': data.get('user_capacity', {}).get('bytes', 0)
    }

    smart_attributes = data.get('ata_smart_attributes', {}).get('table', [])

    for attr in smart_attributes:
        attr_id = attr.get('id')
        raw_val = attr.get('raw', {}).get('value', 0)
        col_name = f'smart_{attr_id}_raw'

        if col_name in nujni_stolpci:
            vsebina[col_name] = raw_val

    for col in nujni_stolpci:
        if col not in vsebina:
            vsebina[col] = 0

    df_test = pd.DataFrame([vsebina])

    df_test['capacity_gigabytes'] = (df_test['capacity_bytes'] / (1024 ** 3)).round(2)
    df_test = df_test.drop(columns=['capacity_bytes'])

    ssd_keywords = ['SSD', 'MTFD', 'SSDSC', '850 PRO', '870 EVO', '860 PRO', '5300']
    df_test['jeSSD'] = 1 if any(k in vsebina['model'].upper() for k in ssd_keywords) else 0

    return df_test

novi_disk = pripravi_podatke_za_model('disk_data_sda.json')
print("Pripravljen DataFrame za tvoj model:")
display(novi_disk)
novi_disk.to_csv('TREUNASscan.csv', index=False)

Pripravljen DataFrame za tvoj model:


,model,smart_5_raw,smart_9_raw,smart_12_raw,smart_187_raw,smart_188_raw,smart_194_raw,smart_199_raw,smart_1_raw,smart_3_raw,...,smart_7_raw,smart_10_raw,smart_190_raw,smart_191_raw,smart_192_raw,smart_193_raw,smart_197_raw,smart_198_raw,capacity_gigabytes,jeSSD
0,WDC WDS500G1R0A-68A4W0,0,3674,11,0,0,171800133667,0,0,0,...,0,0,0,0,0,0,0,0,465.76,0
